## Import Libraries Packages, Downloads

In [774]:
import numpy as np
import pandas as pd


## Process Data

In [775]:
Data = pd.read_csv('NYC_data.csv')


/var/folders/rs/6g2mhpzn6lq_k8wtscfvn5h80000gn/T/ipykernel_17374/2138682545.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Data = pd.read_csv('NYC_data.csv')


In [776]:

def fix_time_formatt(strs):

    for index, s in enumerate(strs): 

        if s in ['1/2 hour', 'Half hour', 'half hour', '1/2hour', '1/2 hr']:
            strs[index] = '0-30'
        if s in ['1 hour','1 hour lat' , '1hr/????mi' ,'1hr 10mins', '1hr late', '1hr', 
                 '1 hours', '1:00 h', '1 hours','one hour' ,'1hr','1 Hour', '1 hr','1 Hour', 
                 '1hour', '1:hr', '1ho', "\\" + '1 hr-', 'hour','1:00 hour' ,'1 hr.', '1 Hour', 
                 '1 hr', '1 hrs', '1 Hour', '1hour', '1:hr', '1hr????', '1hr?????', '1hr???', '1hr??', '1hr?', '1 hourr']:
            strs[index] = '31-60'
        if s in ['45min/1hr','40-1hr' ,'40-1 hour' ,'50min 1hr','50-1hr', 
                 '50-1 hour','45min - 1h' ,'45 -1hour','45mins 1hr', 
                 '45mini/1hr', '45min-1 hr', '45-1hour','45min-1 hr',  
                 '45-1hour', '45/1hour', '45min-1 hr', '45 -1 hour', 
                 '45min -1hr', '45 min-1hr', '45 min-1 h','45min-1hr', 
                 '45-1hr', '45min/1hr', '45 min-1 h']:
            strs[index] = '31 - 60'
        if s in ['1hr 30mins', '1hr /30min','1hour 30mi', '1 hourhalf','1hr/1hr 30',
            '1hr - 11/2','1hr - 1 1/2','1hr-1 1/2', '1.5 hr', '1 hr 30' , '1 hr 30', 
            '1 1/2 hour', '1:30 h', '1hr&30min', '1.5 hour','1 hr 30 mi', '1 hr 30min', 
            '1hr 30 min', '1hr/30min?', '1hr 30min', '1hr/30min']:
            strs[index] = '61-90'
        if s in ['2 hours', '2:hr', '2hr', '2hrs','2 hour', '2 Hours', 
                 '2 hrs','2 Hours', '2hours', '2:hrs', '2hrs????', 
                 '2hrs?????', '2hrs???', '2hrs??', '2hrs?', '2 hr']:
            strs[index] = '91 - 120'
        if s in ['1hr 15mins', '1 hr 15', '1 hour 15', '1hr/15min', '1 hr 15min']:
            strs[index] = '61-90'
        if s in ['1hr 20mins', '1 hr 20', '1hr/20min', '1hr/20min','1hr 20 min',
                 '1hr. 20 mi', '1hr20 mins','1hrs 20mnt', '1hour 20mi','1 hr 20min']:
            strs[index] = '61-90'
        if s in ['1hr 45mins', '1:45hr','1 hr 45', '1hr - 1:45', '1hour45min', 
                 '1 hr 45min','1hr 45 min', '1hr 40 min', '1hr45min', '1hr 45min']:
            strs[index] = '91-120'
        if s in ['4 hrs', '4 hours']:
            strs[index] = '211-240'
        if s in ['3 hrs', '3 hr', '3 hours']:
            strs[index] = '151-180'
        if s in ['30hr', '30hour']:
            strs[index] = '1771 - 1800'

    for index, st in enumerate(strs):
        ns = ''.join(i for i in st if i.isdigit() or i == '-')
        strs[index] = ns


    return strs


def strs_to_int(strs):
    for index, s in enumerate(strs): 
        if s in ['0', '6', '26', '9', '2', '14', '5','5-7','11','5-8','12-18','17','22','5-1','8','24',' 20-20','10-','18',
                 '20-32','20-20','6-7','510', '8-10', '10-18', '19','15-45', '10-12', '15-250','15-','10-14','7-8', '12-15',
                 '18-25', '25-20', '15-20','15-25' ,'13-15', '12-20','1015','15-15', '11', '17','10-30', '1427', '20-', '-10',
                 '2530', '10', '0-10', '10-40','0-15', '15', '25','10-15', '25-35','27','10-11', '15-10','23','2-30', '15-', '-15','25--',
                 '16-30','15-30', '1','20-30', '20-25','28', '20','7-10','26','08','25-', '-25','5-15','35','24','29','15-320','12','4','10-112',
                 '10-20', '25', '25-30','5-10', '20-225','13', '3', '7','20-15', '2030', '25-300',  '1530', '2015', '1435', '0000', '000', 
                 '250', '300']:
            strs[index] = '0-30'
        if s in ['30', '30-40', '30-1','40-1' ,'2025','1-', '50-1','56', '30-' ,'3035','40-50' ,'31-45', '37','41','20-28',
                 '35-45','40-45','40-60','30-60','45-1','40','46-50','20-40', '40-5','50-60','30-50','40-55','31','20-35', '32', '25-40', '21','49',
                 '46-60', '42','30-35', '59', '45-55','4545', '3254', '35-40','39','25-45','39', '3545', '36']:
            strs[index] = '31-60'
        if s in ['30', '30-40','43','49','3040','31-45','30-45', '46-50', '60', '51','34','52-30','50', '40','4560','450',
                 '35','55' , '58', '57','45' ,'46','35','46-60', '50-1', '45-60', '54','30-70', '30-75','100']:
            strs[index] = '31-60'
        if s in ['60-90', '80', '65','90','75']:
            strs[index] = '61-90'
        if s in ['105', '130', '120']:
            strs[index] = '91-120'
        #if s in ['60', '30-40', '31-45', '46-50','46-60']:
    return strs




In [777]:
def process(df): 
    data = df.copy()

    to_binary = {'Yes':1, 'No':0}

    data = data.drop(['Busbreakdown_ID', 'Created_On', 
            'Informed_On','Incident_Number', 
            'Last_Updated_On', 'School_Age_or_PreK', 
            'School_Year', 'Route_Number', 'Schools_Serviced',
            'Run_Type', 'Bus_No', 'Bus_Company_Name'], axis = 1)

    data['Breakdown_or_Running_Late'] = data['Breakdown_or_Running_Late'].map({'Breakdown':1, 'Running Late':0})
    data['Has_Contractor_Notified_Parents'] = data['Has_Contractor_Notified_Parents'].map(to_binary)
    data['Has_Contractor_Notified_Schools'] = data['Has_Contractor_Notified_Schools'].map(to_binary)
    data['Have_You_Alerted_OPT'] = data['Have_You_Alerted_OPT'].map(to_binary)

    data['Occurred_On'] = pd.to_datetime(data['Occurred_On'])
    data['Year'] = data['Occurred_On'].dt.year
    data['Month'] = data['Occurred_On'].dt.month
    data['Day'] = data['Occurred_On'].dt.day
    data['Hour'] = data['Occurred_On'].dt.hour
    data['Mins'] = data['Occurred_On'].dt.minute

    data = data.drop(['Occurred_On'], axis = 1)


    #one hot encode boro
    for boro in data['Boro'].unique():
        data[boro] = (data['Boro'] == boro).astype(int)

    data = data.drop(['Boro'], axis = 1)

    # one hot encode breakdown reason
    for reason in data['Reason'].unique():
        data[reason] = (data['Reason'] == reason).astype(int)

    data = data.drop(['Reason'], axis = 1)

    #fix times 
    data['How_Long_Delayed'] = data['How_Long_Delayed'].fillna(-1)
    
    times_as_strings = data['How_Long_Delayed'].to_string().split('\n')
    times_as_strings = [s.split(None, 1)[1] for s in times_as_strings]
    times_as_strings = fix_time_formatt(times_as_strings)
    times_as_strings = strs_to_int(times_as_strings)
    data['How_Long_Delayed'] = times_as_strings

    #get rid of rows with invalid time entries
    data = data[data['How_Long_Delayed'].isin(['0-30','-1','31-60','61-90','91-120'])]

    return data




In [778]:
df = process(Data)


In [779]:
df.head()

,How_Long_Delayed,Number_Of_Students_On_The_Bus,Has_Contractor_Notified_Schools,Has_Contractor_Notified_Parents,Have_You_Alerted_OPT,Breakdown_or_Running_Late,Year,Month,Day,Hour,...,Heavy Traffic,Flat Tire,Other,Won`t Start,Mechanical Problem,Problem Run,Accident,Late return from Field Trip,Delayed by School,Weather Conditions
0,-1,11,1,0,1,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
1,0-30,2,1,1,0,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
2,0-30,8,1,1,1,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
3,0-30,6,0,0,0,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
4,-1,6,0,0,0,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
